In [1]:
import sys
sys.path.append(r'../../')
from module.Feature.feature import *
import pandas as pd

# 特征提取

在执行本代码块时，需要确保在result/preprocess的文件夹下有feature_70,std_70两个文件夹用来存放数据提取后的结果

In [4]:

fea_col_acc=['wristr_8203_Accel_WR_X_CAL', 'wristr_8203_Accel_WR_Y_CAL','wristr_8203_Accel_WR_Z_CAL']#暂时选这3列
fea_col_gyro=['wristr_8203_Gyro_X_CAL','wristr_8203_Gyro_Y_CAL','wristr_8203_Gyro_Z_CAL']
#分别提取陀螺仪、加速度计特征
for fea_col in [fea_col_gyro,fea_col_acc]:
    if fea_col==fea_col_acc:
        sensor="acc"
    elif fea_col==fea_col_gyro:
        sensor="gyro"
    #elif fea_col==fea_col_mag:
        #sensor="mag"
    else:
        print("错误")
        break
    #分别提取每个病人的特征
    for person in range(1,71,1):
        all_type_data=pd.DataFrame()
        activity_id = 0
        #分别提取每个活动的特征
        for activity in range(1,15,1):
            activity_id = activity_id+1
            person_temp=person
            filefullpath="../../datasets/raw/person{}/{}_session{}_wristr.csv".format(person,person,activity)
            if os.path.exists(filefullpath):
                data=pd.read_csv(filefullpath,low_memory=False)
                data=data[fea_col][2:]
                for col in fea_col[0:]: 
                    data[col] = data[col].astype('float64')
                #融合轴计算
                data["ACCW2"]=np.sqrt(data[fea_col[0]]*data[fea_col[0]]+data[fea_col[1]]*data[fea_col[1]]+data[fea_col[2]]*data[fea_col[2]])
                #标准化
                data[[fea_col[0], fea_col[1], fea_col[2], 'ACCW2']]=data[[fea_col[0], fea_col[1],fea_col[2], 'ACCW2']].apply(lambda x : (x-np.mean(x))/(np.std(x)))
                data = data.values
                datahigh = data.copy()
                databand1 = data.copy()
                databand2 = data.copy()
                databand3 = data.copy()
                for k in range(0,4): 
                    datahigh[:,k] = butter_highpass_filter(data[:,k], 0.3, fs, order=3)
                    databand1[:,k] = butter_bandpass_filter(data[:,k],2, 20, fs, order = 3)
                    databand2[:,k] = butter_bandpass_filter(data[:,k],2, 7, fs, order = 3)
                    databand3[:,k] = butter_bandpass_filter(data[:,k],3, 12, fs, order = 3)
                datahigh = pd.DataFrame(datahigh)
                databand1 = pd.DataFrame(databand1)
                databand2 = pd.DataFrame(databand2)
                databand3 = pd.DataFrame(databand3)
                databand3['label']= activity_id
                data = pd.concat([datahigh,databand1,databand2,databand3],axis = 1)
                data.to_csv("../../result/preprocess/std_70/{}_session{}_wristr_std.csv".format(person,activity),index=False)
                #all_type_data=pd.concat([all_type_data,data])
        print("person:",person)
        #all_type_data.to_csv("E:/DATA/pd_rawdata/std_57/病人{}的14个动作.csv".format(person),index=False)
        #71改变成变量
    for person in range(1,71,1):
        for activity in range(1,15,1):         
            filefullpath="../../result/preprocess/std_70/{}_session{}_wristr_std.csv".format(person,activity)
            if os.path.exists(filefullpath):
                data=pd.read_csv(filefullpath) 
                print(":activity",activity)
                temp_fea=get_fea(data)
                temp_fea['subject_id']=[person for i in range(temp_fea.shape[0])]
                #temp_fea['severity_label']=data['severity_label']
                temp_fea.to_csv("../../result/preprocess/feature_70/activity_fea_{}_{}.csv".format(person,activity),index=False)
                print("finish：",activity)    

    all_data=pd.DataFrame() 
    print("合并所有文件")
    for person in range(1,71,1):
        print(person)
        for activity in range(1,15,1):
            filefullpath="../../result/preprocess/feature_70/activity_fea_{}_{}.csv".format(person,activity)
            if os.path.exists(filefullpath):
                data=pd.read_csv(filefullpath)
                all_data=pd.concat([all_data,data]) 
    all_data=all_data.reset_index(drop=True)
    all_data['severity_level'] = 0
    all_data.to_csv("../../result/preprocess/feature_70/wristr_feature_{}.csv".format(sensor),index=False)   
 

person: 1
person: 2
person: 3
person: 4
person: 5
person: 6
person: 7
person: 8
person: 9
person: 10
person: 11
person: 12
person: 13
person: 14
person: 15
person: 16
person: 17
person: 18
person: 19
person: 20
person: 21
person: 22
person: 23
person: 24
person: 25
person: 26
person: 27
person: 28
person: 29
person: 30
person: 31
person: 32
person: 33
person: 34
person: 35
person: 36
person: 37
person: 38
person: 39
person: 40
person: 41
person: 42
person: 43
person: 44
person: 45
person: 46
person: 47
person: 48
person: 49
person: 50
person: 51
person: 52
person: 53
person: 54
person: 55
person: 56
person: 57
person: 58
person: 59
person: 60
person: 61
person: 62
person: 63
person: 64
person: 65
person: 66
person: 67
person: 68
person: 69
person: 70
:activity 1
len_data-windowsize 4147
finish： 1
:activity 2
len_data-windowsize 4187
finish： 2
:activity 3
len_data-windowsize 4250
finish： 3
:activity 4
len_data-windowsize 5131
finish： 4
:activity 5
len_data-windowsize 5289
finish： 5
:act

finish： 3
:activity 4
len_data-windowsize 5131
finish： 4
:activity 5
len_data-windowsize 5289
finish： 5
:activity 6
len_data-windowsize 4119
finish： 6
:activity 7
len_data-windowsize 4188
finish： 7
:activity 8
len_data-windowsize 5993
finish： 8
:activity 9
len_data-windowsize 5814
finish： 9
:activity 10
len_data-windowsize 4579
finish： 10
:activity 11
len_data-windowsize 4718
finish： 11
:activity 12
len_data-windowsize 6665
finish： 12
:activity 1
len_data-windowsize 4737
finish： 1
:activity 2
len_data-windowsize 4611
finish： 2
:activity 3
len_data-windowsize 4393
finish： 3
:activity 4
len_data-windowsize 4461
finish： 4
:activity 5
len_data-windowsize 4755
finish： 5
:activity 6
len_data-windowsize 4193
finish： 6
:activity 7
len_data-windowsize 4360
finish： 7
:activity 8
len_data-windowsize 4295
finish： 8
:activity 9
len_data-windowsize 11181
finish： 9
:activity 10
len_data-windowsize 5824
finish： 10
:activity 11
len_data-windowsize 8491
finish： 11
:activity 1
len_data-windowsize 4436
fi

# 拼接加速度和陀螺仪的特征

In [5]:
acc_fea=pd.read_csv("../../result/preprocess/feature_70/wristr_feature_acc.csv")
gyro_fea=pd.read_csv("../../result/preprocess/feature_70/wristr_feature_gyro.csv")
#mag_fea=pd.read_csv("E:/pd_pre_label/feature/wristr_feature_mag.csv")

temp=pd.concat([acc_fea[[str(i) for i in range(0,121,1)]],gyro_fea],axis=1)
temp['severity_level'] = 0

temp.shape

temp.columns=[str(i) for i in range(0,242,1)]+['Activity_id', 'subject_id','severity_level']

temp.to_csv("../../result/preprocess/feature_70/final_wristr_fea_acc_gyro_70pd_200hz.csv",index=False)#00000a


# 添加分期标签

In [8]:
import pandas as pd
import re
label=pd.read_excel('../../datasets/raw/label_shimmer3.xlsx')
data = pd.read_csv('../../result/preprocess/feature_70/final_wristr_fea_acc_gyro_70pd_200hz.csv')
fenqi=[]
exportdata=pd.DataFrame()
for i in range(0,29*14,15):
    fenqi.append(label['分期估计'][i])
for i in range(421,439,17):
    fenqi.append(label['分期估计'][i])
for i in range(455,950,19):
    fenqi.append(label['分期估计'][i])

fenqi.append(1)
fenqi.append(2)
fenqi.append(2)
fenqi.append(1)#61
fenqi.append(2)#62~67
fenqi.append(2)
fenqi.append(2)
fenqi.append(2)
fenqi.append(2)
fenqi.append(2)
fenqi.append(1)#68
fenqi.append(2)#69
fenqi.append(3)#70

for i in range(1,71,1):
    temp=data.loc[data['subject_id']==i]
    temp['severity_level']=fenqi[i-1]
    exportdata = exportdata.append(temp, ignore_index=True)
exportdata.to_csv('../../result/preprocess/242D_HC_PD_window300_wristr_acc_gro_200hz.csv',index=None)


C:\Users\13700\AppData\Local\Temp/ipykernel_21028/1982490851.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['severity_level']=fenqi[i-1]
